# Subnetwork Inference

In [1]:
%load_ext autoreload
%autoreload 2

In [33]:
import polars as pl
import numpy as np
import glob

## Experiments using UCI gap datasets

### Reproduce results using the sub-network selection strategy proposed in Daxberger et al., 2020

In [52]:
queries = []
for file in glob.glob("results/wine_*LA*.json"):
    q = pl.scan_ndjson(file)
    queries.append(q)
df = pl.concat(queries)
agg_df = df.groupby(["seed", "split",  "selection_strategy", "subset_size"]).agg(pl.mean("nll")).sort(["seed", "split",  "subset_size", "selection_strategy",]).collect()
agg_df = agg_df.groupby(["split", "selection_strategy", "subset_size"]).agg([pl.mean("nll").alias("nll_mean"), pl.std('nll').alias("nll_std")]).sort(["split",  "subset_size", "nll_mean", "selection_strategy",])

agg_df

split,selection_strategy,subset_size,nll_mean,nll_std
str,str,i64,f64,f64
"""GAP""","""OBD""",600,1.148978,0.005107
"""GAP""","""KFE""",600,1.161739,0.005976
"""GAP""","""LVD""",600,1.173145,0.006701
"""GAP""","""OBD""",1200,1.14775,0.005127
"""GAP""","""KFE""",1200,1.156361,0.004645
"""GAP""","""LVD""",1200,1.165477,0.005821
"""GAP""","""OBD""",1800,1.148506,0.005052
"""GAP""","""KFE""",1800,1.153049,0.005269
"""GAP""","""LVD""",1800,1.156799,0.005195


In [53]:
queries = []
for file in glob.glob("results/kin8nm_*LA*.json"):
    q = pl.scan_ndjson(file)
    queries.append(q)
df = pl.concat(queries)
agg_df = df.groupby(["seed", "split",  "selection_strategy", "subset_size"]).agg(pl.mean("nll")).sort(["seed", "split",  "subset_size", "selection_strategy",]).collect()
agg_df = agg_df.groupby(["split", "selection_strategy", "subset_size"]).agg([pl.mean("nll").alias("nll_mean"), pl.std('nll').alias("nll_std")]).sort(["split",  "subset_size", "nll_mean", "selection_strategy",])

agg_df

split,selection_strategy,subset_size,nll_mean,nll_std
str,str,i64,f64,f64
"""GAP""","""OBD""",600,0.185231,0.010533
"""GAP""","""KFE""",600,0.19355,0.011578
"""GAP""","""LVD""",600,0.200172,0.012568
"""GAP""","""OBD""",1200,0.170907,0.009811
"""GAP""","""KFE""",1200,0.180126,0.0111
"""GAP""","""LVD""",1200,0.184411,0.011777
"""GAP""","""OBD""",1800,0.165734,0.009329
"""GAP""","""KFE""",1800,0.172372,0.010824
"""GAP""","""LVD""",1800,0.173089,0.010619


### Results of using pruning methods as a sub-network selection strategy


### Results of using KFAC approximated sub-network selection strategy

## Conclusion